In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip freeze

In [13]:
num_classes=62  #although only 34 classes are used 62 have been put so as to make minimal changes in the class mapping 
img_size=28
def img_label(data_path, num_classes=62):
    data = pd.read_csv(data_path, header=None)
    data_rows = len(data)
    
    imgs = np.transpose(data.values[:,1:].reshape(data_rows, img_size, img_size, 1),axes=[0,2,1,3])  #values in csv were such that the image needs to be transposed before training
    
    labels = to_categorical(data.values[:,0], num_classes)
    
    return imgs/255, labels

In [14]:
model=Sequential()
model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = 4, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(62, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# model.summary()

In [15]:
data_generator_aug=ImageDataGenerator(rotation_range=30,validation_split=0.1)

In [16]:
data_generator=ImageDataGenerator(validation_split=0.1)

In [17]:
X,Y=img_label('/content/drive/MyDrive/mosiac/filename.csv',62)

In [18]:
checkpoint = ModelCheckpoint('/content/drive/My Drive/modified_weights-{epoch:03d}.h5',
                                 monitor='val_acc',
                                 verbose=0,
                                 save_best_only=False,
                                 mode='auto')

In [20]:
training_data_generator = data_generator_aug.flow(X, Y,batch_size=300, subset='training')
validation_data_generator = data_generator.flow(X, Y,batch_size=300,subset='validation')
history = model.fit_generator(training_data_generator, steps_per_epoch=100, epochs=40, validation_data=validation_data_generator,validation_steps=1,callbacks=[checkpoint])

Epoch 1/40


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


100/100 [==============================] - 11s 110ms/step - loss: 0.3945 - accuracy: 0.8899 - val_loss: 1.1639 - val_accuracy: 0.6733
Epoch 2/40
100/100 [==============================] - 15s 147ms/step - loss: 0.3759 - accuracy: 0.8938 - val_loss: 1.0521 - val_accuracy: 0.6500
Epoch 3/40
100/100 [==============================] - 11s 107ms/step - loss: 0.3532 - accuracy: 0.8985 - val_loss: 1.0990 - val_accuracy: 0.6633
Epoch 4/40
100/100 [==============================] - 11s 108ms/step - loss: 0.3322 - accuracy: 0.9037 - val_loss: 0.7797 - val_accuracy: 0.7267
Epoch 5/40
100/100 [==============================] - 13s 125ms/step - loss: 0.3209 - accuracy: 0.9105 - val_loss: 0.9766 - val_accuracy: 0.6767
Epoch 6/40
100/100 [==============================] - 11s 108ms/step - loss: 0.3026 - accuracy: 0.9114 - val_loss: 0.8423 - val_accuracy: 0.7367
Epoch 7/40
100/100 [==============================] - 11s 108ms/step - loss: 0.3028 - accuracy: 0.9133 - val_loss: 0.8710 - val_accuracy: 0.7

In [21]:
model.save('model5.h5')

In [23]:
from google.colab import files
files.download("model5.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>